# PINN Testing

In [ ]:
import os
from dotenv import load_dotenv
import sys

In [ ]:
load_dotenv()

lib_dir = os.environ.get("LOCAL_LIB_DIR")
sys.path.append(lib_dir)

data_dir_airfoil = os.environ.get("LOCAL_DATA_DIR_AIRFOIL")
model_dir_airfoil = os.path.join(data_dir_airfoil, "models")

In [ ]:
import numpy as np
import torch
import yaml
from airfoil_pinn import AirfoilPINN
from naca4digit_airfoil import Naca4DigitAirfoil
from matplotlib import pyplot as plt
import utils

In [ ]:
with open("config_airfoil.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
device = torch.device("cpu")

In [ ]:
# Define airfoil parameters (example: NACA 2412)
chord = 1.0
m = 0.02  # maximum camber
p = 0.4   # position of maximum camber
t = 0.12  # maximum thickness
num_points = 100
angle_of_attack = 10.0

airfoil = Naca4DigitAirfoil(chord, m, p, t, angle_of_attack, num_points)

# testing domain
domain = utils.Domain2D(
        x_min=config["domain"]["x_min"], 
        x_max=config["domain"]["x_max"], 
        y_min=config["domain"]["y_min"],
        y_max=config["domain"]["y_max"])

In [ ]:
def show_airfoil():
  fig, ax = plt.subplots(figsize=(12, 7))
  airfoil.plot(ax=ax)
  domain.plot(ax=ax)
  plt.show()

show_airfoil()

In [ ]:
center_x, center_y = (domain.x_max + domain.x_min) / 2, (domain.y_max + domain.y_min) / 2

# Translate to the origin
airfoil.translate(-center_x, -center_y)
domain.translate(-center_x, -center_y)

# Scale the airfoil and the domain
width = domain.x_max - domain.x_min
height = domain.y_max - domain.y_min
# scaling_factor_x = 1 / width
# scaling_factor_y = 1 / height
scaling_factor = 1 / max(width, height)

# airfoil.scale(sx=scaling_factor_x, sy=scaling_factor_y)
# domain.scale(sx=scaling_factor_x, sy=scaling_factor_y)

airfoil.scale(sx=scaling_factor, sy=scaling_factor)
domain.scale(sx=scaling_factor, sy=scaling_factor)

fig, ax = plt.subplots(figsize=(12, 7))
airfoil.plot(ax=ax)
domain.plot(ax=ax)

plt.show()

In [ ]:
# load model
model_name = config["model_name"]
model_checkpoint_num = config["model_checkpoint_num"]

pinn = AirfoilPINN.load_from_checkpoint_for_testing(model_dir_airfoil, model_name, model_checkpoint_num).to(device)

In [ ]:
x_min = domain.x_min
x_max = domain.x_max
y_min = domain.y_min
y_max = domain.y_max

Nx = config["Nx"]
Ny = config["Ny"]

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)

x_test = np.linspace(x_min, x_max, Nx)
y_test = np.linspace(y_min, y_max, Ny)

X, Y = np.meshgrid(x_test, y_test)

X_flat = X.flatten()
Y_flat = Y.flatten()

input = np.vstack([X_flat, Y_flat]).T

input_tensor = torch.tensor(input, dtype=torch.float32)

input_tensor = input_tensor.to(device)
output_uvp = pinn(input_tensor)

u_test = output_uvp[:, 0].to("cpu").detach().numpy()
v_test = output_uvp[:, 1].to("cpu").detach().numpy()
p_test = output_uvp[:, 2].to("cpu").detach().numpy()

velocity_norm = np.linalg.norm([u_test, v_test], axis=0)

# min & max values of the model outputs
velocity_norm = np.array(velocity_norm)
min_velocity_norm = velocity_norm.min()
max_velocity_norm = velocity_norm.max()

min_pressure = np.array(p_test).min()
max_pressure = np.array(p_test).max()

In [ ]:
# domain.scale(sx=1/scaling_factor_x, sy=1/scaling_factor_y)
# airfoil.scale(sx=1/scaling_factor_x, sy=1/scaling_factor_y)

domain.scale(sx=1/scaling_factor, sy=1/scaling_factor)
airfoil.scale(sx=1/scaling_factor, sy=1/scaling_factor)

airfoil.translate(center_x, center_y)
domain.translate(center_x, center_y)

x_min = domain.x_min
x_max = domain.x_max
y_min = domain.y_min
y_max = domain.y_max

Nx = config["Nx"]
Ny = config["Ny"]

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)

x_test = np.linspace(x_min, x_max, Nx)
y_test = np.linspace(y_min, y_max, Ny)

X, Y = np.meshgrid(x_test, y_test)

U = u_test.reshape(Nx, Ny)
V = v_test.reshape(Nx, Ny)

N = velocity_norm.reshape(Nx, Ny)

U = np.ma.masked_where(N < 0.03, U)
V = np.ma.masked_where(N < 0.03, V)

fig, ax = plt.subplots(figsize=(12, 7))

strm = ax.streamplot(X, Y, U, V, color=N, linewidth=1, cmap='jet', density=2, arrowstyle='->', arrowsize=1)

airfoil.plot(ax=ax)

fig.colorbar(strm.lines, ax=ax, label='Velocity magnitude')

ax.set_title('Streamlines')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

plt.show()

In [ ]:
X, Y = np.meshgrid(x_test, y_test)

N = velocity_norm.reshape(Nx, Ny)

fig, ax = plt.subplots(figsize=(12, 7))

contour = ax.contourf(X, Y, N, levels=500, cmap='jet')

plt.colorbar(contour, ax=ax, label='Velocity Norm')

airfoil.plot(ax=ax)

ax.set_title('Velocity')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()

plt.show()

In [ ]:
X, Y = np.meshgrid(x_test, y_test)

P = p_test.reshape(Nx, Ny)

fig, ax = plt.subplots(figsize=(12, 7))

contour = ax.contourf(X, Y, P, levels=500, cmap='jet')

plt.colorbar(contour, ax=ax, label='Pressure Distribution')

airfoil.plot(ax=ax)

ax.set_title('Pressure')
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()

plt.show()